# Create Views for the best Location

This section attempts to visulise a suitable place might be to open a restaurant.<br>The existing crimes, restaurants and hot spot records are be use.

In [ ]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!pip install geopy  # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


# Matplotlib and associated plotting modules
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import matplotlib.ticker as ticker
%matplotlib inline

# notice: installing seaborn might takes a few minutes
!pip install seaborn
import seaborn as sns
sns.set_style("darkgrid")

!pip install folium==0.8.3  # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from folium import plugins
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

from PIL import Image # converting images into arrays

# install wordcloud
!pip install wordcloud==1.4.1

# import package and its set of stopwords
from wordcloud import WordCloud, STOPWORDS

print('Libraries imported.')

# Read CSV File Toronto Top 10 Type Dinas

In [ ]:
slDinaToronto = pd.read_csv('./data/Toronto_dinasafe_Top10.csv')

In [ ]:
slDinaToronto.shape

In [ ]:
slDinaToronto['TYPE'].value_counts().to_frame().head()

## Establishment type ie restaurant, mobile cart
https://open.toronto.ca/dataset/dinesafe/

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(30, 10))
sns.countplot(x='TYPE', data = slDinaToronto, order = slDinaToronto['TYPE'].value_counts().index)
ax.set_title("Establishment type")
for t in ax.patches:
    if (np.isnan(float(t.get_height()))):
        ax.annotate('', (t.get_x(), 0))
    else:
        ax.annotate((format(int(t.get_height()), ',d')), (t.get_x(), t.get_height()*1.01))

plt.xticks(rotation='vertical')    
plt.show();

# Toronto Cultural Hot Spot
https://open.toronto.ca/dataset/cultural-spaces/

##### Performance
Spaces in which performing arts (dance, music, theatre, etc.) creation or presentation takes place.  Includes but is not limited to:<br>
·         Dance studios
·         Recording studios
·         Rehearsal spaces
·         Theatres
·         Music halls / live music spaces 
·         Public squares
·         Restaurants and bars with stages / performance spaces
·         Churches
·         Band shells / amphitheatres / outdoor spaces that can accommodate performances (eg. Shakespeare in the Park, Manifesto Dance)
·         Multipurpose spaces with specific accommodations for performances (eg. lighting, audience seating, etc.)

##### Exhibition / Visual Arts
Spaces in which visual arts creation or presentation can take place, in addition to pure exhibition space.  Includes but is not limited to:<br>
·         Art galleries
·         Museums
·         Visual artist studios
·         Photography studios
·         Exhibition space in coffee shops

##### Screen Based
Spaces for the production and presentation of multimedia screen-based arts including digital, photography, and film.  Includes but is not limited to:<br>
·         Film studios
·         Movie theatres
·         Digital media labs (eg. ProTech Media Centre)
·         Production/post production facilities

##### Library
·         Toronto Public Library facility with physical space for cultural activity.<br>

##### Multipurpose
Spaces that are not purpose-built and can house a range of cultural activity across disciplines. Includes but is not limited to:<br>
·         Parks
·         Community Centres
·         Recreation Centres
·         Public squares
·         Churches
·         Schools

##### Heritage
·         Museums
·         Facilities where heritage activity takes place (historical societies, archives, community museums etc.)

## Load CSV File Toronto Culture Hot Spot

In [ ]:
slHLocToronto = pd.read_csv('./data/Toronto_Culture_HotSpot.csv')

In [ ]:
slHLocToronto.TYPE.value_counts().to_frame()

## Use geopy library to get the latitude and longitude values of Toronto, CA.

In [ ]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="Jupyter")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, CA are {}, {}.'.format(latitude, longitude))

## Create a map of Toronto with Restaurants and Culture Hot Spot.

In [ ]:
slHLocToronto = slHLocToronto.sample(frac=1).head(500)

In [ ]:
slDinaToronto = slDinaToronto.sample(frac=1).head(500)

In [ ]:
dfToronto = pd.concat([slDinaToronto, slHLocToronto])

In [ ]:
dfTorontoType = dfToronto[['TYPE','colorGrp']].groupby(['TYPE']).first()

### Visualize establishment in Torronto

In [ ]:
str = ''

# create map of Toronto, CA using latitude and longitude values
map_markets = folium.Map(location=[latitude, longitude], zoom_start=12)


# add markers to map
for lat, lng, name, types, colo in zip(slHLocToronto['LATITUDE'], slHLocToronto['LONGITUDE'], slHLocToronto['NAME'], slHLocToronto['TYPE'], slHLocToronto['colorGrp']):
            label = '{}, {}, {};{}'.format(name, types, lat, lng)
            colo = colo.replace('[','').replace(']','').split(',')
            label = folium.Popup(label, parse_html=True)
            folium.CircleMarker(
                [lat, lng],
                radius=3,
                popup=label,
                color=f'#{int(colo[0]):02x}{int(colo[1]):02x}{int(colo[2]):02x}',
                fill=True,
                fill_color=f'#{int(colo[0]):02x}{int(colo[1]):02x}{int(colo[2]):02x}',
                fill_opacity=0.7,
                parse_html = False).add_to(map_markets)  

# add markers to map
for lat, lng, name, types, colo in zip(slDinaToronto['LATITUDE'], slDinaToronto['LONGITUDE'], slDinaToronto['NAME'], slDinaToronto['TYPE'], slDinaToronto['colorGrp']):
            label = '{}, {}, {};{}'.format(name, types, lat, lng)
            label = folium.Popup(label, parse_html=True)
            colo = colo.replace('[','').replace(']','').split(',')
            folium.CircleMarker(
                [lat, lng],
                radius=3,
                popup=label,
                color=f'#{int(colo[0]):02x}{int(colo[1]):02x}{int(colo[2]):02x}',
                fill=True,
                fill_color=f'#{int(colo[0]):02x}{int(colo[1]):02x}{int(colo[2]):02x}',
                fill_opacity=0.7,
                parse_html = False).add_to(map_markets)  

for types, colo in zip(dfTorontoType.index, dfTorontoType['colorGrp']):
    colo = colo.replace('[','').replace(']','').split(',')
    str += '<span style="color:{};">&nbsp; - {}</span><br>'.format(f'#{int(colo[0]):02x}{int(colo[1]):02x}{int(colo[2]):02x}',types)  
 
        
legend_html = """
     <div style="position: fixed; top: 400px; left: 1350px; width: 60x; height: 110x; border:1px solid grey; border-radius: 5px; z-index:9999; 
     background-color:white; opacity: .7; border-radius:6px; font-size:13px; font-weight: bold;">
     &nbsp; <b>{title}<b/> <br>
     {itm_txt}
     </ul>
     </div> """.format( title = "Legend", itm_txt= str)

map_markets.get_root().html.add_child(folium.Element( legend_html ))    

map_markets.save('./data/dinahotspot_map.html')
            
map_markets

## Create a police boundery map of Toronto with Restaurants and Culture Hot Spot.

In [ ]:
dfToronto = pd.read_csv('./data/Toronto_dinasafe.csv')

In [ ]:
x = dfToronto[['AREA_LONG_CODE','TYPE']].groupby(['AREA_LONG_CODE'])

In [ ]:
z = x.count().reset_index()

In [ ]:
# create a map
m = folium.Map(location=[43.653963, -79.387207], zoom_start=11)


## add chloropleth layer
folium.Choropleth(
    geo_data='./data/Police Boundaries Data.geojson',
    name='Number of Estabiliment',
    data=z,
    columns=['AREA_LONG_CODE','TYPE'],
    key_on='feature.properties.AREA_LONG_CODE',
    fill_color='YlOrBr',
    fill_opacity=0.4,
    line_opacity=1.0,
    legend_name='Number of Estabiliment'
).add_to(m) 

# enable layers to be turned in or out
folium.LayerControl().add_to(m)

m.save('./data/crimedivision_map.html')

m

## The most used Type in Toronto

In [ ]:
stopwords = set(STOPWORDS)

In [ ]:
dfTorontoNAME = dfToronto.NAME.apply(lambda x: x.replace(' ',','))
dfTorontoNAME.head()

In [ ]:
# Start with one review:
text = " ".join(review for review in dfTorontoNAME)

# lower max_font_size, change the maximum number of word and lighten the background:
wordcloud = WordCloud(stopwords=stopwords, max_font_size=60, max_words=200, background_color="white").generate(text)
plt.figure(figsize=(50, 20))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

# Visualising The Toronto Major Crime Indicators (MCI) Data

For a better understanding of the Toronto MCI Crime Dataset I will visualise the data.

In [ ]:
# Import the MCI in the DataFrame
dfMCI = pd.read_csv('./data/Toronto_MCI.csv', parse_dates=['reportedda'])

In [ ]:
# Import the MCI in the DataFrame
dfMCI10 = pd.read_csv('./data/Toronto_MCI_Top10.csv', parse_dates=['reportedda'])

In [ ]:
dfMCI.head()

### Create colours per crime/month/hour

Colour for each crime/month/hour; used for bar an maps

In [ ]:
clist = [(0, "red"), (0.125, "red"), (0.25, "orange"), (0.5, "green"), 
         (0.7, "green"), (0.75, "blue"), (1, "blue")]

rvb = colors.LinearSegmentedColormap.from_list("", clist)

N = 24
x = np.arange(N).astype(float)

### Number of Crimes per month

In [ ]:
months = ['January','February','March','April','May','June','July','August','September','October','November','December']

# Group the Crimes per month and produce a bar chart
fig = dfMCI.groupby([dfMCI['reportedda'].dt.month], sort=True).count().plot(y = 'offence', 
                                             kind='bar',
                                             figsize=(20,8),
                                             width=0.9,
                                             color=rvb(x/N))
ax = fig.axes # main axes
ax.set_xticklabels(months)

plt.xlabel('Month')
plt.ylabel('Count of offence')
plt.title('Count of Cases Per Month', loc='left', fontsize=18)

### Number of crimes occuring in each hour

In [ ]:
# Group the Crimes per month and produce a bar chart
fig = dfMCI.groupby([dfMCI['reportedda'].dt.hour], sort=True).count().plot(y = 'offence', 
                                             kind='bar',
                                             figsize=(20,8),
                                             width=0.9,
                                             color=rvb(x/N))
plt.xlabel('Hour')
plt.ylabel('Count of offence')
plt.title('Count of Cases Per Hour', loc='left', fontsize=18)

### Create a Map of the Top 10 Crimes

In [ ]:
# Create a list of the most commonly crimes
top_crimes = dfMCI10[['offence', 'Division']].groupby(
    ['offence']).count().sort_values('Division', ascending=False)[:10].axes[0].tolist()

In [ ]:
top_crimes

### Create a legend for map

In [ ]:
dfMCI10Offence = dfMCI10[['offence','colorGrp']].groupby(['offence']).first()

In [ ]:
# Create a new data frame with just the top 10 crimes
df_top_crimes = dfMCI10[dfMCI10['offence'].isin(top_crimes)].copy()

In [ ]:
df_top_crimes.head()

In [ ]:
str = ''

# Define the world map centered around Toronto, CA with a higher zoom level
crime_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# Instantiate a feature group for the incidents in the dataframe
offence = folium.map.FeatureGroup()

# loop through the August crimes and add each to the incidents feature group
for lat, lng, prem, off, nbh, rep3, div, colo in zip(df_top_crimes.Lat, df_top_crimes.Long
                         , df_top_crimes.premisetyp, df_top_crimes.offence, df_top_crimes.Neighbourh
                         , df_top_crimes.reported_3, df_top_crimes.Division, df_top_crimes.colorGrp):
    
    label = '{}, {}, {}, {}, {}'.format(div, prem, off, rep3, nbh)
    colo = colo.replace('[','').replace(']','').split(',')
    
    offence.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=3,
            popup=label,
            color=f'#{int(colo[0]):02x}{int(colo[1]):02x}{int(colo[2]):02x}',
            fill=True,
            fill_color=f'#{int(colo[0]):02x}{int(colo[1]):02x}{int(colo[2]):02x}',
            fill_opacity=0.6))  
    
# add incidents to map
crime_map.add_child(offence)


for offence, colo in zip(dfMCI10Offence.index, dfMCI10Offence['colorGrp']):
    colo = colo.replace('[','').replace(']','').split(',')
    str += '<span style="color:{};">&nbsp; - {}</span><br>'.format(f'#{int(colo[0]):02x}{int(colo[1]):02x}{int(colo[2]):02x}',offence)  
 
        
legend_html = """
     <div style="position: fixed; top: 250px; left: 1550px; width: 60x; height: 110x; border:1px solid grey; border-radius: 5px; z-index:9999; 
     background-color:white; opacity: .7; border-radius:6px; font-size:8px;">
     &nbsp; <b>{title}<b/> <br>
     {itm_txt}
     </ul>
     </div> """.format( title = "Legend", itm_txt= str)

crime_map.get_root().html.add_child(folium.Element( legend_html ))    

crime_map.save('./data/crimedina_map.html')

# display world map
crime_map

In the above map has visualised the crimes data for month, October in Toronto, CA.

In this view are visulised same data as a Cluster Map.

In [ ]:
mc = MarkerCluster()

# Define the world map centered around Toronto
crimedina_cluster = folium.Map(location=[latitude, longitude], zoom_start=12)

#creating a Marker for each point in df_sample. Each point will get a popup with their zip
for row in df_top_crimes.itertuples():
    mc.add_child(folium.Marker(location=[row.Lat, row.Long], popup=row.offence))


# add Cluster
crimedina_cluster.add_child(mc)

for row in slDinaToronto.itertuples():
    mc.add_child(folium.Marker(location=[row.LATITUDE, row.LONGITUDE], popup=row.TYPE))

# add Cluster
crimedina_cluster.add_child(mc)

crimedina_cluster.save('./data/crimedina_clustermap.html')

crimedina_cluster